# Exercises

Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [3]:
df1 = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

In [4]:
df2 = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

In [5]:
df

DataFrame[source_id: string, source_username: string]

In [6]:
df1

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]

In [7]:
df2

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

## 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [8]:
df.write.json("sources_json", mode="overwrite")

In [9]:
df.write.csv("sources_csv", mode="overwrite")

> Inspect your folder structure. What do you notice?

In [10]:
# the files have no extension and are named just as written "sources_json and sources_csv"

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [11]:
df.show(2, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
only showing top 2 rows



In [12]:
df.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [13]:
df1.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [14]:
df1.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [15]:
df1.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [16]:
df2.show(2, vertical=True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
only showing top 2 rows



In [17]:
df2.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [18]:
df1 = df1.withColumnRenamed("SLA_due_date", "case_due_date")

In [19]:
df1.groupBy("case_closed", "case_late").count().show()

+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [20]:
df1 = df1.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

df1.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [21]:
df1 = df1.withColumn("council_district", col("council_district").cast("string"))

In [22]:
print("--- Before handling dates")
df1.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
df1 = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
df1.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



AnalysisException: cannot resolve 'case_opened_date' given input columns: [source_id, source_username];
'Project [source_id#16, source_username#17, to_timestamp('case_opened_date, Some(M/d/yy H:mm)) AS case_opened_date#327]
+- Relation [source_id#16,source_username#17] csv


************************************

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [23]:
df1 = (
    df1.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

df1.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

df1.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+----------------+----------------+--------+--------------+-------------+
|case_closed|case_opened_date|case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+----------------+----------------+--------+--------------+-------------+
|       true|     1/1/18 0:42|    1/1/18 12:29|    null|          null|         null|
|       true|     1/1/18 0:46|     1/3/18 8:11|    null|          null|         null|
|       true|     1/1/18 0:48|     1/2/18 7:57|    null|          null|         null|
|       true|     1/1/18 1:29|     1/2/18 8:13|    null|          null|         null|
|       true|     1/1/18 1:34|    1/1/18 13:29|    null|          null|         null|
+-----------+----------------+----------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+----------------+----------------+--------+--------------+-------------+
|case_closed|case_opened_date|case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+----------------

In [24]:
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

AnalysisException: Table or view not found: df; line 3 pos 5;
'GlobalLimit 15
+- 'LocalLimit 15
   +- 'Sort ['days_past_due DESC NULLS LAST], true
      +- 'Project ['DATEDIFF('current_timestamp, 'case_due_date) AS days_past_due#447]
         +- 'Filter NOT 'case_closed
            +- 'UnresolvedRelation [df], [], false


In [25]:
# Another way to do it:
(
    df.select(datediff(current_timestamp(), 'case_due_date')
    .alias('days_past_due'))
    .where(df.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
)

AnalysisException: cannot resolve 'case_due_date' given input columns: [source_id, source_username];
'Project [datediff(current_timestamp(), 'case_due_date) AS days_past_due#448]
+- Relation [source_id#16,source_username#17] csv


### 2. How many Stray Animal cases are there?

In [26]:
df1.filter(df1.service_request_type == 'Stray Animal').count()

26760

In [27]:
(
    df1.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [28]:
(
    df1.filter(df1.dept_division == 'Field Operations')
    .filter(df1.service_request_type != 'Officer Standby')
    .count()
)

113902

In [29]:
# Another way to do it
(
    df1.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

113902

### 4. Convert the council_district column to a string column.

In [30]:
# Already done in the data prep

### 5. Extract the year from the case_closed_date column.

In [31]:
df1.select('case_closed_date', year('case_closed_date')).show(5)

+----------------+----------------------+
|case_closed_date|year(case_closed_date)|
+----------------+----------------------+
|    1/1/18 12:29|                  null|
|     1/3/18 8:11|                  null|
|     1/2/18 7:57|                  null|
|     1/2/18 8:13|                  null|
|    1/1/18 13:29|                  null|
+----------------+----------------------+
only showing top 5 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [32]:
(
    df1.withColumn('num_hours_late', df1.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

### 7. Join the case data with the source and department data.

In [33]:
df1 = (
    df1
    # left join on dept_division
    .join(df2, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(df2.dept_division)
    .drop(df2.dept_name)
    .drop(df1.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

df1.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | null                 
 days_to_closed       | null                 
 case_lifetime        | null                 
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:4

In [34]:
df.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [35]:
df.sort(col('source_id').desc()).show()

+---------+-----------------+
|source_id|  source_username|
+---------+-----------------+
|  yh24110| Yojani Hernandez|
|  yc16753|    Yvonne Casias|
|  vk26526|    Vincent Kosub|
|  vb22265| Vanessa Burciaga|
|  ts15690|  Thelma Martinez|
|  sw26367|Samantha Wickwire|
| svcCRMSS|         svcCRMSS|
| svcCRMLS|         svcCRMLS|
| svcCFlag|         CityFlag|
|  sv24848|  Salvador Valdez|
|  ss26317|     Sakib Shaikh|
|  ss21394|  Shirley Sanchez|
|  ss09159|   Sylvia Sanchez|
|  sp26368|  Steven Peterson|
|  sg26196|   Samantha Garza|
|  sg22264|  Steven Martinez|
|  ru26699|      Raul Ubides|
|  rs16746|Ronald S. Sorrell|
|  rb05270|       Rene Bomar|
|  ps01944|   Paula Stallcup|
+---------+-----------------+
only showing top 20 rows



In [36]:
(
    df1.join(df, "source_id", "left")
    .sort(col("source_username"))
    .show(5, vertical=True)
)

-RECORD 0------------------------------------
 source_id            | af26445              
 case_id              | 1014220226           
 case_opened_date     | 2/5/18 15:07         
 case_closed_date     | 2/5/18 17:59         
 case_due_date        | 6/13/18 15:07        
 case_late            | false                
 num_days_late        | -127.88046299999999  
 case_closed          | true                 
 service_request_type | Base/Pavement Repair 
 SLA_days             | 128.0                
 case_status          | Closed               
 request_address      | BAYWATER DR and L... 
 council_district     | 7                    
 case_age             | null                 
 days_to_closed       | null                 
 case_lifetime        | null                 
 department           | Trans & Cap Impro... 
 dept_subject_to_SLA  | true                 
 source_username      | Alex Franklin        
-RECORD 1------------------------------------
 source_id            | af26445   

### 8. Are there any cases that do not have a request source?

In [37]:
(
    df1.select(df1.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [38]:
df1.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



### 9. What are the top 10 service request types in terms of number of requests?

In [39]:
(
    df1.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



### 10.What are the top 10 service request types in terms of average days late?
#### - just the late cases
#### - for the late cases:
####   - what is the average number of days late by request type?

In [40]:
(
    df1.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|2      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [41]:
(
    df1.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

AnalysisException: cannot resolve 'dept_name' given input columns: [SLA_days, case_age, case_closed, case_closed_date, case_due_date, case_id, case_late, case_lifetime, case_opened_date, case_status, council_district, days_to_closed, department, dept_subject_to_SLA, num_days_late, request_address, service_request_type, source_id];
'Aggregate ['dept_name], ['dept_name, avg(num_days_late#41) AS days_late#1158, count(num_days_late#41) AS n_cases_late#1160L]
+- Filter case_late#265: boolean
   +- Project [case_id#36, case_opened_date#37, case_closed_date#38, case_due_date#199, case_late#265, num_days_late#41, case_closed#250, service_request_type#44, SLA_days#45, case_status#46, source_id#47, request_address#48, council_district#293, case_age#328, days_to_closed#344, case_lifetime#361, department#674, (dept_subject_to_SLA#83 = YES) AS dept_subject_to_SLA#693]
      +- Project [case_id#36, case_opened_date#37, case_closed_date#38, case_due_date#199, case_late#265, num_days_late#41, case_clo

In [42]:
df.groupby('dept_name').count().show(truncate=False)

AnalysisException: cannot resolve 'dept_name' given input columns: [source_id, source_username];
'Aggregate ['dept_name], ['dept_name, count(1) AS count#1164L]
+- Relation [source_id#16,source_username#17] csv


### 12. How do number of days late depend on department and request type?

In [43]:
(
    df1.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

AnalysisException: cannot resolve 'standardized_dept_name' given input columns: [SLA_days, case_age, case_closed, case_closed_date, case_due_date, case_id, case_late, case_lifetime, case_opened_date, case_status, council_district, days_to_closed, department, dept_subject_to_SLA, num_days_late, request_address, service_request_type, source_id];
'Aggregate ['standardized_dept_name, service_request_type#44], ['standardized_dept_name, service_request_type#44, avg(num_days_late#41) AS days_late#1184, count(1) AS n_cases#1186L]
+- Filter case_closed#250: boolean
   +- Project [case_id#36, case_opened_date#37, case_closed_date#38, case_due_date#199, case_late#265, num_days_late#41, case_closed#250, service_request_type#44, SLA_days#45, case_status#46, source_id#47, request_address#48, council_district#293, case_age#328, days_to_closed#344, case_lifetime#361, department#674, (dept_subject_to_SLA#83 = YES) AS dept_subject_to_SLA#693]
      +- Project [case_id#36, case_opened_date#37, case_close